In [ ]:
import pandas as pd
import numpy as np
import pulp # this is a popular optimization library


In [ ]:
# Functions to find the best team

def load_and_prepare_data(file_path):
    # Load the data from the Excel file.
    # This is the function to change if you have a better format for swimmers and their best times
    data = pd.read_excel(file_path) 

    # Just put in a very high time for any missing strokes
    data['Time'] = data['Time'].fillna(data['Time'].max())

    return data

def find_optimal_team(data):
    # Define strokes
    strokes = ['Free', 'Back', 'Breast', 'Fly']

    # Create a list of all unique swimmers
    swimmers = data['Swimmer Num'].unique()

    # Create a dictionary for the swimmer times
    swimmer_times = {(row['Swimmer Num'], row['Stroke']): row['Time'] for idx, row in data.iterrows()}

    # Create the optimization problem
    prob = pulp.LpProblem("MedleyRelayTeamOptimization", pulp.LpMinimize)

    # Define decision variables given a list of swimmers and strokes
    decision_vars = pulp.LpVariable.dicts("SwimmerStroke", ((swimmer, stroke) 
                                                            for swimmer in swimmers for stroke in strokes), 
                                          cat='Binary')

    # Objective function: minimize total time given a dict of times
    prob += pulp.lpSum([swimmer_times[(swimmer, stroke)] * decision_vars[(swimmer, stroke)] 
                        for swimmer in swimmers for stroke in strokes])

    # Constraints: Each stroke must be assigned exactly one swimmer
    for stroke in strokes:
        prob += pulp.lpSum([decision_vars[(swimmer, stroke)] for swimmer in swimmers]) == 1

    # Constraints: Each swimmer can only be assigned to one stroke
    for swimmer in swimmers:
        prob += pulp.lpSum([decision_vars[(swimmer, stroke)] for stroke in strokes]) <= 1

    # Solve the problem
    prob.solve()

    # Get the results
    best_team = []
    for swimmer in swimmers:
        for stroke in strokes:
            if pulp.value(decision_vars[(swimmer, stroke)]) == 1:
                best_team.append((swimmer, stroke, swimmer_times[(swimmer, stroke)]))

    return best_team

def generate_top_n_teams(file_path, n):
    data = load_and_prepare_data(file_path)
    all_teams = []

    for i in range(n):
        best_team = find_optimal_team(data)
        if not best_team:
            break

        all_teams.append(best_team)

        # Remove the swimmers in the best team from the dataset
        selected_swimmers = [swimmer for swimmer, stroke, time in best_team]
        data = data[~data['Swimmer Num'].isin(selected_swimmers)]

    return all_teams

# Print the results in a user-friendly format
def print_teams(teams):
    for i, team in enumerate(teams, start=1):
        print(f"#{i} Medley Team:")
        for swimmer, stroke, time in team:
            print(f"Swimmer: {swimmer}, Stroke: {stroke}, Time: {time:.2f}")
        total_time = sum(time for swimmer, stroke, time in team)
        print(f"Total Time: {total_time:.2f}\n")



In [ ]:
# File paths
file_path = 'swim-opt.xlsx'

# Generate top N teams
n = 2
top_teams = generate_top_n_teams(file_path, n)

In [ ]:
print_teams(top_teams)